# Imports

In [1]:
from dqn_agent import DQNAgent
from ppo_agent import PPOAgent
from ffnn_agent import FFNNAgent
from data_processing import preprocess_all_data, load_preprocessed_dataset, get_activity, get_column_units
from data_processing import load_preprocessed_dataset, get_biased_feature_percentage, train_test_split_data, get_xy_from_data, get_activity, get_column_units
from visualize import visualize_results, visualize_training, visualize_preprocessed_data
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import json
import pandas as pd

# Load Data

In [ ]:
# Load the preprocessed dataset
df = load_preprocessed_dataset()

# Print unique values of Activity ID
unique_activity_ids = df['Activity ID'].unique()
print("Unique Activity IDs:", unique_activity_ids)
print(df.head())

# Get percentage of biased feature
biased_feature_percentage = get_biased_feature_percentage(df, print_result=False)

# Split into 70% train, 15% val, 15% test with stratification
df_train, df_temp = train_test_split(
    df, test_size=0.30, stratify=df['Sex - Female'], random_state=42
)
df_val, df_test = train_test_split(
    df_temp, test_size=0.50, stratify=df_temp['Sex - Female'], random_state=42
)

# Get X and y data from training, validation, and testing sets
x_train, y_train = get_xy_from_data(df_train)
x_val, y_val = get_xy_from_data(df_val)
x_test, y_test = get_xy_from_data(df_test)

# Make y_train 2d
y_train = y_train.values.reshape(-1, 1)
y_val = y_val.values.reshape(-1, 1)
y_test = y_test.values.reshape(-1, 1)

print(y_train.shape)
print(x_train.shape)

# Initialize system

In [ ]:
continuous_columns = [
    'Timestamp', 'Heart Rate', 'Hand Sensor - Temperature',
    'Hand Sensor - Accelerometer - X', 'Hand Sensor - Accelerometer - Y',
    'Hand Sensor - Accelerometer - Z', 'Hand Sensor - Gyroscope - X',
    'Hand Sensor - Gyroscope - Y', 'Hand Sensor - Gyroscope - Z',
    'Hand Sensor - Magnetometer - X', 'Hand Sensor - Magnetometer - Y',
    'Hand Sensor - Magnetometer - Z', 'Chest Sensor - Temperature',
    'Chest Sensor - Accelerometer - X', 'Chest Sensor - Accelerometer - Y',
    'Chest Sensor - Accelerometer - Z', 'Chest Sensor - Gyroscope - X',
    'Chest Sensor - Gyroscope - Y', 'Chest Sensor - Gyroscope - Z',
    'Chest Sensor - Magnetometer - X', 'Chest Sensor - Magnetometer - Y',
    'Chest Sensor - Magnetometer - Z', 'Ankle Sensor - Temperature',
    'Ankle Sensor - Accelerometer - X', 'Ankle Sensor - Accelerometer - Y',
    'Ankle Sensor - Accelerometer - Z', 'Ankle Sensor - Gyroscope - X',
    'Ankle Sensor - Gyroscope - Y', 'Ankle Sensor - Gyroscope - Z',
    'Ankle Sensor - Magnetometer - X', 'Ankle Sensor - Magnetometer - Y',
    'Ankle Sensor - Magnetometer - Z', 'Height', 'Weight',
    'Resting HR', 'Max HR'
]

discrete_columns = [
    'Age', 'Sex - Female'
]

#discrete action size columns
dqn_config = {
    'state_size': 5,  
    'action_size': len(discrete_columns),  
    'hidden_size': 64,
    'lr': 1e-3,
    'gamma': 0.99,
    'batch_size': 32,
    'memory_size': 10000,
    'epsilon_start': 1.0,
    'epsilon_min': 0.01,
    'epsilon_decay': 0.995
}


#continuous
ppo_config = {
    'state_size': 5,  
    'action_size': len(continuous_columns),   
    'hidden_size': 64,
    'lr': 3e-4,
    'gamma': 0.99,
    'clip_epsilon': 0.2,
    'update_epochs': 4,
    'batch_size': 64,
    'c1': 0.5,
    'c2': 0.01
}
classes = [1, 2, 3, 17, 16, 13, 4, 7, 6]
ffnn_config = {
    'input_size': df.shape[1] - 1,
    'hidden_sizes': [64, 64],
    'output_size': len(classes),
    'learning_rate': 0.001,
    'batch_size': 32,
    'epochs': 100,
    'type': 'classification',
    'classes': classes
}

synthetic_data_amount = 100
num_episodes = 10000

dqn_agent = DQNAgent(**dqn_config)
ppo_agent = PPOAgent(**ppo_config)
ffnn_agent = FFNNAgent(**ffnn_config)
ffnn_agent_og = FFNNAgent(**ffnn_config)

# Training

In [ ]:
import numpy as np
import pandas as pd
import torch
import json

def evaluate_ffnn(ffnn_agent, data, labels):
    # Ensure data and labels are NumPy arrays
    if isinstance(data, (pd.DataFrame, pd.Series)):
        data = data.to_numpy()
    if isinstance(labels, (pd.Series, pd.DataFrame)):
        labels = labels.to_numpy().reshape(-1)

    # Convert to tensors
    data_tensor = torch.tensor(data, dtype=torch.float32)
    labels_tensor = torch.tensor(labels, dtype=torch.long).view(-1)  # <- ensures 1D shape


    device = next(ffnn_agent.model.parameters()).device
    data_tensor = data_tensor.to(device)
    labels_tensor = labels_tensor.to(device)

    # Model prediction
    ffnn_agent.model.eval()
    with torch.no_grad():
        outputs = ffnn_agent.model(data_tensor)
        predicted_classes = torch.argmax(outputs, dim=1)

    # Sanity check
    if predicted_classes.shape != labels_tensor.shape:
        print("Shape mismatch:", predicted_classes.shape, labels_tensor.shape)
        labels_tensor = labels_tensor.view(-1)
        predicted_classes = predicted_classes.view(-1)

    # Compute accuracy
    correct = (predicted_classes == labels_tensor).sum().item()
    total = labels_tensor.size(0)
    accuracy = correct / total

    print(f"Correct: {correct}, Total: {total}, Accuracy: {accuracy:.4f}")
    return accuracy




def generate_state(df, mf_ratio, n_samples):
    timestamp = np.random.uniform(df['Timestamp'].min(), df['Timestamp'].max())
    male_female_ratio = mf_ratio 
    num_samples = n_samples
    age = np.random.uniform(df['Age'].min(), df['Age'].max())
    activity_id = np.random.choice([1, 2, 3, 17, 16, 13, 4, 7, 6])
    return np.array([timestamp, male_female_ratio, num_samples, age, activity_id])


def compute_mini_reward(synthetic_data, mf_ratio):
    column_std = np.std(synthetic_data, axis=0).mean()
    gaussian_penalty = np.exp(-((mf_ratio - 0.5) ** 2) / 0.1)
    return column_std + gaussian_penalty


def train_agents(x_train, y_train, x_val, y_val, x_test, y_test,
                 dqn_agent, ppo_agent, ffnn_agent, episodes=num_episodes,
                 save_path='training_metrics.json'):

    # Normalize y arrays to shape (n, 1)
    for var in ['y_train', 'y_val', 'y_test']:
        val = locals()[var]
        if isinstance(val, pd.Series):
            locals()[var] = val.to_numpy().reshape(-1, 1)
        elif isinstance(val, pd.DataFrame):
            locals()[var] = val.to_numpy()

    rewards = []
    accuracies = []
    test_accuracies = []
    synthetic_data = []
    synthetic_labels = []

    # Initial male-female ratio
    sex_female_idx = x_train.columns.get_loc('Sex - Female')
    mf_ratio = np.mean(x_train.iloc[:, sex_female_idx])
    state = generate_state(x_train, mf_ratio, 0)

    for episode in range(episodes):
        if synthetic_data:
            synthetic_array = np.array(synthetic_data)
            if synthetic_array.ndim == 1:
                synthetic_array = synthetic_array.reshape(1, -1)
            combined_array = np.vstack([x_train.to_numpy(), synthetic_array])
            combined = pd.DataFrame(combined_array, columns=x_train.columns)
        else:
            combined = x_train.copy()

        sex_female_idx = combined.columns.get_loc('Sex - Female')
        mf_ratio = np.mean(combined.iloc[:, sex_female_idx])

        discrete_action = np.array(dqn_agent.predict(state), ndmin=1)
        continuous_action = np.array(ppo_agent.predict(state), ndmin=1)

        synthetic_row = np.zeros(x_train.shape[1])
        discrete_indices = x_train.columns.get_indexer(discrete_columns)
        continuous_indices = x_train.columns.get_indexer(continuous_columns)
        synthetic_row[discrete_indices] = discrete_action
        synthetic_row[continuous_indices] = continuous_action

        synthetic_data.append(synthetic_row)
        synthetic_labels.append(state[4])  # activity ID

        mini_reward = compute_mini_reward(np.array(synthetic_data), mf_ratio)
        done = len(synthetic_data) >= synthetic_data_amount

        if done:
            synthetic_data_np = np.array(synthetic_data)
            synthetic_labels_np = np.array(synthetic_labels).reshape(-1, 1)

            combined_data = np.vstack([x_train.to_numpy(), synthetic_data_np])
            combined_labels = np.vstack([y_train, synthetic_labels_np])

            # Shuffle combined training data
            indices = np.arange(combined_data.shape[0])
            np.random.shuffle(indices)
            combined_data = combined_data[indices]
            combined_labels = combined_labels[indices]

            # Train FFNN
            ffnn_agent.train(combined_data, combined_labels)


            accuracy = evaluate_ffnn(ffnn_agent, x_val, y_val)
            test_accuracy = evaluate_ffnn(ffnn_agent, x_test, y_test)

            reward = accuracy
            reward += mini_reward
            accuracies.append(accuracy)
            test_accuracies.append(test_accuracy)

            print(f"Episode {episode + 1}/{episodes} | Reward: {reward:.4f} | Validation Accuracy: {accuracy:.4f} | Test accuracy: {test_accuracy:.4f}")

            synthetic_data = []
            synthetic_labels = []
        else:
            reward = mini_reward

        next_state = generate_state(x_train, mf_ratio, len(synthetic_data) + 1)
        dqn_agent.learn(state, discrete_action, reward, next_state, done)
        ppo_agent.learn(state, continuous_action, reward, next_state, done)

        rewards.append(reward)
        state = next_state

    metrics = {
        'rewards': rewards,
        'accuracies': accuracies,
        'test_accuracies': test_accuracies
    }

    with open(save_path, 'w') as f:
        json.dump(metrics, f)
    print(f"Metrics saved to {save_path}")

    return metrics

# Example usage
results = train_agents(x_train, y_train, x_val, y_val, x_test, y_test, dqn_agent, ppo_agent, ffnn_agent)
dqn_agent.save("dqn_trained_model.pth")
ppo_agent.save("ppo_trained_model.pth")


# Testing


In [ ]:

accuracy = evaluate_ffnn(ffnn_agent, x_test, y_test)
print(f"Test Results | Reward: {reward:.4f} | Accuracy: {accuracy:.4f}")


ffnn_agent_og.train(x_train.to_numpy(), y_train)
og_accuracy = evaluate_ffnn(ffnn_agent_og, x_test, y_test)
print(f"No Synthetic Data Test Accuracy: {og_accuracy}")


# Visualize results

In [6]:
# visualize_training([], [])
# visualize_results(dqn_agent, ppo_agent, df)
# visualize_preprocessed_data(df)